In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from arguments import parse_args
from main import train, get_data
from dataset import LocalizationDataset
from torch.utils.data import DataLoader
import os
from cust_model import Tracker, TrackingDataset
import pandas as pd
import numpy as np
import torch

In [3]:
amzn_data = pd.read_csv("AMZN28.csv").set_index("DateTime").drop('Unnamed: 0', axis = 1, inplace = False)

In [4]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
args = AttrDict({"num_trajs":1000,
        "sl":100,
        "batch_size":128,
        "num_particles":30,
        "window_length":6,
        "h":64,
        "emb_obs":23,
        "emb_act":32,
        "dropout":0.5,
        "obs_num":5,
        "resamp_alpha":0.5,
        "bp_length":10,
        "clip":1.0,
        "lr":0.0005,
        "bpdecay":0.1})

In [5]:
amznnp = amzn_data.to_numpy()

In [6]:
inlen = args.sl*(len(amznnp) // args.sl)
inputs_window = np.array([amznnp[i-args.window_length:i] for i in range(args.window_length,len(amznnp))])[:inlen]
outputs = amznnp[args.window_length:inlen+args.window_length,0]

In [7]:
inputs_prebatch = inputs_window.reshape(inlen // args.sl, args.sl, -1)
outputs_prebatch = outputs.reshape(inlen // args.sl, args.sl)

In [8]:
train_dataset = TrackingDataset(inputs_prebatch, outputs_prebatch)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size,
                            num_workers=8, pin_memory=True, shuffle=False)
model = Tracker(args)
optimizer = torch.optim.RMSprop(model.parameters(), lr=args.lr)

In [9]:
cnt = 0
best_eval = 1000
from tqdm import tqdm

print("starting epochs")
for epoch in tqdm(range(1)):
    print("going to train")
    model.train() #just a toggle switch

    print("starting iterations")
    for iteration, data in enumerate(train_loader):
        cnt = cnt + 1

        output, window = data
        model.zero_grad()
        loss, log_loss, particle_pred = model.step(
            window, output, args)
        loss.backward()

        if args.clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()

starting epochs


  0%|          | 0/1 [00:00<?, ?it/s]

going to train
starting iterations


/Users/michaelsprintson/Documents/nosync.nosync/pfrnns/cust_model.py:97: UserWarning: Using a target size (torch.Size([128, 100])) that is different to the input size (torch.Size([128, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  l2_pred_loss = torch.nn.functional.mse_loss(pred, gt_normalized, reduction='none') * bpdecay_params
  0%|          | 0/1 [00:24<?, ?it/s]


RuntimeError: The size of tensor a (100) must match the size of tensor b (128) at non-singleton dimension 1